In [1]:
import pandas as pd
import os
from src.data_matching.main import get_edgelist

In [2]:
path="/mnt/c/Users/nkired/Documents/data-matching/tests/input_data"
# for file in os.listdir(path):
#     base_name=os.path.basename(file).split(".")[0]
#     path_file=os.path.join(path,file)
#     df=pd.read_csv(path_file)
#     reduced_df=df.sample(int(0.25*len(df)))
#     base_new_name=base_name+"_small.csv"
#     reduced_df.to_csv(os.path.join(path,base_new_name),index=False)


In [3]:
from src.data_matching.EmbDI.edgelist import EdgeList

def get_edgelist(input_file, out_dir, export: bool =True, overwrite: bool = True, dry_run: bool = False):
    """Translate an input CSV file into an edgelist."""
    dfpath = input_file
    base_name = os.path.basename(input_file).replace(".csv", ".txt")
    edgefile = os.path.join(out_dir, base_name)
    info_file = None
    df = pd.read_csv(dfpath, low_memory=False)
    pref = ["3#__tn", "3$__tt", "5$__idx", "1$__cid"]
    el = EdgeList(df, edgefile, pref, info_file, flatten=True)
    if dry_run:
        if export:
            el.convert_to_dict()
            gdict = el.convert_to_dict()
            print("el", el.convert_to_dict())
            print("gdict", gdict)
            g_nx = nx.from_dict_of_lists(gdict)
            n, _ = osp.splitext(edgefile)
            nx_fname = n + ".nx"
            pkl_fname = n + ".pkl"
            if overwrite:
                with open(nx_fname, "wb") as nx_file:
                    pickle.dump(g_nx, nx_file)
                with open(pkl_fname, "wb") as pkl_file:
                    pickle.dump(gdict, pkl_file)


In [4]:
input_path="/mnt/c/Users/nkired/Documents/data-matching/tests/input_data"
output_path="/mnt/c/Users/nkired/Documents/data-matching/tests/output_data"
for file in os.listdir(input_path):
    path_file=os.path.join(input_path,file)
    if "small" in file:
        get_edgelist(path_file, output_path)
        

340it [00:00, 2333.50it/s]


806it [00:00, 2619.71it/s]


1086it [00:00, 2388.34it/s]


750it [00:00, 1710.18it/s]


654it [00:00, 1765.01it/s]


...

13980it [00:04, 3064.97it/s]


1250it [00:00, 1528.78it/s]


10it [00:00, 948.14it/s]


5813it [00:04, 1354.35it/s]


2906it [00:02, 1390.19it/s]